## Análise de Cancelamento de Assinaturas

A mineração de dados oferece diversas vantagens para a análise descritiva, proporcionando insights valiosos a partir de conjuntos de dados extensos. Ao utilizar técnicas de mineração de dados para a análise descritiva, é possível identificar padrões, tendências e relações significativas nos dados, facilitando a compreensão de características fundamentais e fornece uma visão mais abrangente do comportamento dos dados. Ademais, a mineração de dados permite a identificação rápida de outliers e a visualização de informações de maneira intuitiva, simplificando a interpretação e comunicação dos resultados. Em suma, as vantagens incluem a capacidade de extrair conhecimento significativo, descobrir padrões ocultos e tomar decisões informadas com base na análise descritiva dos dados.






In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:


import pandas as pd

tabela = pd.read_csv("/content/drive/MyDrive/Python/PYTHON/assinaturas /cancelamentos.csv")
tabela = tabela.drop("CustomerID", axis=1)
display(tabela)

,idade,sexo,tempo_como_cliente,frequencia_uso,ligacoes_callcenter,dias_atraso,assinatura,duracao_contrato,total_gasto,meses_ultima_interacao,cancelou
0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,932.00,17.0,1.0
1,65.0,Female,49.0,1.0,10.0,8.0,Basic,Monthly,557.00,6.0,1.0
2,55.0,Female,14.0,4.0,6.0,18.0,Basic,Quarterly,185.00,3.0,1.0
3,58.0,Male,38.0,21.0,7.0,7.0,Standard,Monthly,396.00,29.0,1.0
4,23.0,Male,32.0,20.0,5.0,8.0,Basic,Monthly,617.00,20.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
881661,42.0,Male,54.0,15.0,1.0,3.0,Premium,Annual,716.38,8.0,0.0
881662,25.0,Female,8.0,13.0,1.0,20.0,Premium,Annual,745.38,2.0,0.0
881663,26.0,Male,35.0,27.0,1.0,5.0,Standard,Quarterly,977.31,9.0,0.0
881664,28.0,Male,55.0,14.0,2.0,0.0,Standard,Quarterly,602.55,2.0,0.0


In [ ]:
# identificando e removendo valores vazios
display(tabela.info())
tabela = tabela.dropna()
display(tabela.info())

In [ ]:
# quantas pessoas cancelaram e não cancelaram
display(tabela["cancelou"].value_counts()) # número absoluto de clientes que cancelaram (1.0) e que não cancelaram (0.0)
display(tabela["cancelou"].value_counts(normalize=True).map("{:.1%}".format)) #porcentagem de clientes que cancelaram e que não cancelaram

1.0    499998
0.0    381666
Name: cancelou, dtype: int64

1.0    56.7%
0.0    43.3%
Name: cancelou, dtype: object

In [ ]:
display(tabela["duracao_contrato"].value_counts(normalize=True)) # porcentagem de clientes com relação a duração do contrato escolhido
display(tabela["duracao_contrato"].value_counts()) # número absoluto dos clientes com relação a duração do contrato escolhido

In [ ]:
# analisando o contrato mensal
display(tabela.groupby("duracao_contrato").mean(numeric_only=True))
# descobrimos aqui que a média de cancelamentos é 1, ou seja, praticamente todos os contratos mensais cancelaram (ou todos)
# observar as variáveis ligacoes_callcenter e dias_atraso

In [ ]:
# então descobrimos que contrato mensal é ruim, vamos tirar ele e continuar analisando
tabela = tabela[tabela["duracao_contrato"]!="Monthly"]
display(tabela)
display(tabela["cancelou"].value_counts())
display(tabela["cancelou"].value_counts(normalize=True).map("{:.1%}".format))

#observe como a porcentagem de cancelamentos cai para 46.1%

In [ ]:
# chegamos agora em menos da metade de pessoas cancelando, vamos continuar analisando
display(tabela["assinatura"].value_counts(normalize=True))
display(tabela.groupby("assinatura").mean(numeric_only=True))
# vemos que assinatura é quase 1/3, 1/3, 1/3
# os cancelamentos são em média bem parecidos, então fica difícil tirar alguma conclusão baseada na média

In [ ]:
import plotly.express as px

tabela['rotulos_cancelou'] = tabela['cancelou'].map({0: 'Não Cancelou', 1: 'Cancelou'})

# Vamos criar  para cada variável em relação à variável 'cancelou'
variaveis = ['idade', 'sexo', 'tempo_como_cliente', 'frequencia_uso', 'ligacoes_callcenter', 'dias_atraso', 'assinatura', 'duracao_contrato']

for variavel in variaveis:
  fig = px.ba(tabela, x=variavel, color='rotulos_cancelou',
  title=f'Histograma de {variavel} em relação a Cancelou',
  labels={'rotulos_cancelou': ''})
 # fig.update_layout(barmode='overlay') # Adiciona os histogramas sobrepostos
 # fig.update_traces(opacity=0.75) # Ajusta a opacidade para melhor visualização
  fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# com os graficos a gente consegue descobrir muita coisa:
# dias atraso acima de 20 dias, 100% cancela
# ligações call center acima de 5 todo mundo cancela

tabela = tabela[tabela["ligacoes_callcenter"]<5]
tabela = tabela[tabela["dias_atraso"]<=20]
display(tabela)
display(tabela["cancelou"].value_counts())
display(tabela["cancelou"].value_counts(normalize=True).map("{:.1%}".format))

# se resolvermos isso, já caímos para 18% de cancelamento
# é claro que 100% é utópico, mas com isso já temos as principais causas (ou talvez 3 das principais):
# - forma de contrato mensal
# - necessidade de ligações no call center
# - atraso no pagamento